# bokeh in the notebook: managing resources lifecycle 


## About this notebook

This notebook belongs to a series of small projects which aim is to evaluate the [Jupyter](http://jupyter.org/) ecosystem for science experiments control. The main idea is use the _Juypter notebook_ as a convergence platform in order to offer a fully featured environment to scientists. 

## About bokeh

Experiments control requires both static and dynamic (i.e live) data visualization. Since Jupyter doesn't provide any 'official' data visualization solution, we need to select one. Among the available solutions, [bokeh](http://bokeh.pydata.org/en/latest) presents the highest potential for our application.

Bokeh as been selected for its:
1. [built-in notebook integration](http://bokeh.pydata.org/en/latest/docs/user_guide/notebook.html)
2. built-in [data streaming](http://bokeh.pydata.org/en/latest/docs/reference/models/sources.html#bokeh.models.sources.ColumnDataSource.patch) [features](http://bokeh.pydata.org/en/latest/docs/reference/models/sources.html#bokeh.models.sources.ColumnDataSource.stream) for live plots update 
3. ability to add [custom or specialized behaviors](http://bokeh.pydata.org/en/latest/docs/user_guide/interaction/callbacks.html) in response to property changes and other events
4. [graphics quality](http://bokeh.pydata.org/en/latest/docs/gallery.html#gallery)

Have a look to this [quickstart](http://bokeh.pydata.org/en/latest/docs/user_guide/quickstart.html) for a bokeh overview.

## Topic of the day

The following content tries to point out a problem we faced while evaluating bokeh in the jupyter notebook. To summarize, let's say that bokeh works really well and fulfills our requirements but it currently (*) has some side-effects that make things diverge in terms of performances and memory consumption. 

(*) true for bokeh version <= 0.12.9

So let's see what we are talking about...  

### BokehSession class

** THIS CODE HAS EVOLVED BUT THE FOLLOWING CODE REMAINS VALID - SEE LAST VERSION ON THE [GITHUB REPO](https://github.com/nleclercq/jupyter-for-controls/blob/master/bokeh-data-streaming-for-notebook/session.py)**

This is the super class of any _session_ we open on the `BokehServer` singleton. 

Our model is based on a _'one session per notebook cell'_ approach. It means that each _session_ is tightly linked to a particular cell. This is a good thing cause we obviously want the bokeh plots to appear as _outputs_ of the cell from which they've created. More generally, we'll certainly want every _output_ related to session to be routed to its associated cell. That's the next 'topic of the day' we'll treat. 

In [ ]:
import logging
from uuid import uuid4
from threading import Lock, Condition

class BokehSession(object):
    
    __repo__ = dict()
    __repo_lock__ = Lock()

    __logger__ = logging.getLogger('BokehSession')
    __logger__.setLevel(logging.DEBUG)
    
    def __init__(self, uuid=None):
        # session identifier
        self._uuid = uuid if uuid else str(uuid4())
        # associated bokeh application 
        self._app = None #TODO: is this really useful?
        # the associated bokeh document (for experts only)
        self._doc = None
        # periodic callback period in seconds - defaults to None (i.e. disabled)
        self._callback_period = None
        # periodic activity enabled?
        self._suspended = True
        # is this session closed?
        self._closed = False
        # close existing session: this is a way to avoid leaks & resources waste
        self.__close_existing_session(self._uuid)
        # insert new session into the repo
        with BokehSession.__repo_lock__:
            BokehSession.__repo__[self._uuid] = self
        BokehSession.print_repository_status()
            
    def __close_existing_session(self, uuid):
        with BokehSession.__repo_lock__:
            try:
                session = BokehSession.__repo__[uuid]
                session.close()
            except KeyError:
                pass
            except Exception as e:
                BokehSession.__logger__.error(e)
            finally:
                try:
                    del BokehSession.__repo__[uuid]
                except KeyError:
                    pass
                except Exception as e:
                    BokehSession.__logger__.error(e)
    
    def _on_session_created(self, app, doc):
        self._app = app
        self._doc = doc
        self.setup_document()

    def _on_session_destroyed(self):
        pass

    @property
    def uuid(self):
        return self._uuid
    
    @property
    def ready(self):
        return self._doc is not None

    @property
    def document(self):
        return self._doc

    @property
    def bokeh_session_id(self):
        return self._doc.session_context.id if self._doc else None

    @property
    def application(self):
        return self._app
        
    @property
    def suspended(self):
        return self._suspended

    @property 
    def callback_period(self):
        """return the (periodic) callback period in seconds or None (i.e. disabled)"""
        return self._callback_period

    @callback_period.setter 
    def callback_period(self, ucbp):
        """set the (periodic) callback period in seconds or None to disable the callback"""
        self._callback_period = max(0.1, ucbp) if ucbp is not None else None

    def open(self):
        """open the session"""
        BokehServer.open_session(self)

    def close(self):
        """close the session"""
        #TODO: async close required but might not be safe!
        self.pause()
        self.safe_document_modifications(self.__cleanup)
        
    def __cleanup(self):
        """asynchronous close"""
        #TODO: async close required but might not be safe!
        try:
            if self._doc:
                self._doc.clear()
            BokehServer.close_session(self)
        except Exception as e:
            BokehSession.__logger__.error(e)
        finally:
            self._closed = True
            with BokehSession.__repo_lock__:
                del BokehSession.__repo__[self._uuid]
        
    def setup_document(self):
        """give the session a chance to setup the freshy created bokeh document"""
        pass

    def periodic_callback_enabled(self):
        """return True if the periodic callback is enabled, return False otherwise"""
        return not self.callback_period == None 
    
    def periodic_callback(self):
        """periodic callback (default impl. does nothing)"""
        pass
  
    def start(self):
        """start the periodic activity (if any)"""
        self.resume()
      
    def stop(self):
        """stop the periodic activity (if any)"""
        self.pause()
        
    def pause(self):
        """suspend the (periodic) callback"""
        self.__set_callback_period(None)
        self._suspended = True
    
    def resume(self):
        """resume the (periodic) callback"""
        self.__set_callback_period(self.callback_period)
        self._suspended = False

    def update_callback_period(self, cbp):
        self.callback_period = cbp
        self.__set_callback_period(cbp)

    def __set_callback_period(self, cbp):
        try:
            self.document.remove_periodic_callback(self.periodic_callback)
        except:
            pass
        if cbp is not None:
            self.document.add_periodic_callback(self.periodic_callback, max(100, int(1000. * cbp)))

    def timeout_callback(self, cb, tmo):
        """call the specified callback after expiration of the specified timeout (in seconds)"""
        if self.ready:
            self._doc.add_timeout_callback(cb, int(1000. * tmo))

    def safe_document_modifications(self, cb):
        """call the specified callback in the a context in which the session document is locked"""
        if self.ready:
            self._doc.add_next_tick_callback(cb)
            
    def __repr__(self):
        return "BokehSession:{}:{}".format(self._uuid, ('closed' if self._closed else 'opened'))
        
    @staticmethod
    def close_all():
        with BokehSession.__repo_lock__:
            for s in BokehSession.__repo__.values():
                try:
                    s.close()
                except Exception as e:
                    BokehSession.__logger__.error('failed to close BokehSession:{}'.format(s.uuid))
                
    @staticmethod
    def print_repository_status():
        with BokehSession.__repo_lock__:
            if len(BokehSession.__repo__):
                BokehSession.__logger__.info('BokehSession.repo. contains {} session(s):'.format(len(BokehSession.__repo__)))
                for s in BokehSession.__repo__.values():
                    BokehSession.__logger__.info('- {}'.format(s))
            else:
                BokehSession.__logger__.info('BokehSession.repo is empty')

### BokehServer class

** THIS CODE HAS EVOLVED BUT THE FOLLOWING CODE REMAINS VALID - SEE LAST VERSION ON THE [GITHUB REPO](https://github.com/nleclercq/jupyter-for-controls/blob/master/bokeh-data-streaming-for-notebook/session.py)**

Embedded bokeh server. Private singleton. 
Implementation derived from [this bokeh example](https://github.com/bokeh/bokeh/blob/master/examples/howto/server_embed/notebook_embed.ipynb).


In [ ]:
import logging
from collections import deque
from threading import Lock

from bokeh.io import output_notebook 
from bokeh.resources import Resources, INLINE
from bokeh.plotting import show
from bokeh.application import Application
from bokeh.application.handlers import FunctionHandler

class BokehServer(object):
    
    __sessions__ = deque()
    __sessions_lock__ = Lock()

    __logger__ = logging.getLogger('BokehServer')
    __logger__.setLevel(logging.ERROR)

    @staticmethod
    def open_session(new_session):
        BokehServer.__logger__.debug("BokehServer.open_session <<")
        logging.getLogger('bokeh.server.util').setLevel(logging.ERROR) #TODO: tmp stuff
        output_notebook(Resources(mode='inline', components=["bokeh", "bokeh-gl"]), hide_banner=True)
        app = Application(FunctionHandler(BokehServer.__session_entry_point))
        with BokehServer.__sessions_lock__:
            session_info = {'session':new_session, 'application':app}
            BokehServer.__sessions__.appendleft(session_info)
        show(app)
        BokehServer.__logger__.debug("BokehServer.open_session >>")
        
    @staticmethod
    def __session_entry_point(doc):
        BokehServer.__logger__.debug("BokehServer.__session_entry_point <<")
        try:
            BokehServer.__logger__.debug('BokehServer.__session_entry_point [doc:{}] <<'.format(id(doc)))
            with BokehServer.__sessions_lock__:
                session_info = BokehServer.__sessions__.pop()
            session = session_info['session']
            BokehServer.__logger__.info("BokehServer.__session_entry_point:opening session {}".format(session))
            session._on_session_created(session_info['application'], doc)
            BokehServer.__logger__.debug('BokehServer.__session_entry_point [doc:{}] >>'.format(id(doc)))
        except Exception as e:
            BokehServer.__logger__.error(e)
        finally:
            BokehServer.__logger__.debug("BokehServer.__session_entry_point >>")
            return doc
            
    @staticmethod
    def close_session(session):
        assert(isinstance(session, BokehSession))
        #TODO: is the document.clear called from the BokeSession.__cleanup is enough to release 
        #TODO: every single resource associated with the session?

### MySession class
A user specialization of the `BokehSession`.

In [ ]:
import numpy as np

from bokeh.plotting import figure
from bokeh.plotting.figure import Figure
from bokeh.models.glyphs import Rect
from bokeh.models import ColumnDataSource
from bokeh.models.widgets import Slider
from bokeh.layouts import layout, widgetbox

class MySession(BokehSession):
    
    def __init__(self, uuid=None):
        BokehSession.__init__(self, uuid)
        self.callback_period = 1.
        self._np = 100
        self._widgets_layout = None
        columns = dict()
        columns['x'] = self._gen_x_scale()
        columns['y'] = self._gen_y_random_data()
        self._cds = ColumnDataSource(data=columns)

    def _gen_x_scale(self):
        """x data"""
        return np.linspace(1, self._np, num=self._np, endpoint=True)
    
    def _gen_y_random_data(self):
        """y data"""
        return np.random.rand(self._np)
    
    def __on_update_period_change(self, attr, old, new):
        """called when the user changes the refresh period using the dedicated slider"""
        self.update_callback_period(new)
        
    def __on_num_points_change(self, attr, old, new):
        """called when the user changes the number of points using the dedicated slider"""
        self._np = int(new)

    def setup_document(self):
        """setup the session model then return it"""
        # a slider to control the update period
        rrs = Slider(start=0.25, 
                     end=2, 
                     step=0.25, 
                     value=self.callback_period, 
                     title="Updt.period [s]",)
        rrs.on_change("value", self.__on_update_period_change)
        # a slider to control the number of points
        nps = Slider(start=0, 
                     end=1000, 
                     step=10, 
                     value=self._np, 
                     title="Num.points")
        nps.on_change("value", self.__on_num_points_change)
        # the figure and its content
        p = figure(plot_width=650, plot_height=200)
        p.toolbar_location = 'above'
        p.line(x='x', y='y', source=self._cds, color="navy", alpha=0.5)
        # widgets are placed into a dedicated layout
        self._widgets_layout = widgetbox(nps, rrs)
        # arrange all items into a layout then return it as the session model
        self.document.add_root(layout([[self._widgets_layout, p]]))
        # start periodic activity
        self.start()

    def periodic_callback(self):
        """periodic activity"""
        self._cds.data.update(x=self._gen_x_scale(), y=self._gen_y_random_data())

### Why is it so important to name a session?

To be written... 

In [ ]:
s1 = MySession('s1')
s1.open()

In [ ]:
s1.close()

In [ ]:
BokehSession.print_repo_status()